In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
from geopy.distance import great_circle
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

import time

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server
name_project

'project_repartition_v3.0/output_repar_01-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['07-03--1',
 '09-03--2',
 '10-03--3',
 '11-03--4',
 '12-03--5',
 '13-03--6',
 '14-03--7',
 '16-03--8',
 '17-03--9',
 '18-03--10',
 '19-03--11',
 '20-03--12',
 '21-03--13',
 '22-03--14',
 '23-03--15',
 '24-03--16',
 '25-03--17',
 '26-03--18',
 '27-03--19',
 '28-03--20',
 '29-03--21',
 '31-03--22',
 '32-03--23',
 '34-03--24',
 '36-03--25',
 '37-03--26',
 '38-03--27',
 '39-03--28',
 '40-03--29',
 '41-03--30',
 '43-03--31',
 '45-03--32',
 '46-03--33',
 '47-03--34',
 '48-03--35',
 '49-03--36',
 '52-03--37',
 '53-03--38',
 '54-03--39',
 '07-04--40',
 '08-04--41',
 '09-04--42',
 '10-04--43',
 '11-04--44',
 '12-04--45',
 '14-04--46',
 '15-04--47',
 '16-04--48',
 '18-04--49',
 '19-04--50',
 '20-04--51',
 '21-04--52',
 '22-04--53',
 '23-04--54',
 '24-04--55',
 '25-04--56',
 '26-04--57',
 '27-04--58',
 '28-04--59',
 '29-04--60',
 '30-04--61',
 '31-04--62',
 '32-04--63',
 '33-04--64',
 '36-04--65',
 '39-04--66',
 '40-04--67',
 '43-04--68',
 '45-04--69',
 '46-04--70',
 '47-04--71',
 '48-04--72',
 

In [7]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic['name_CC']
#dir_CC = dir_CC_workspace+'CC/CC_40_prewhiten/'

In [8]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [9]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [10]:
info_basic_bi['r_max'] = {}

In [11]:
stainfo_all = pd.read_excel(dir_project+info_basic['stalistname'],sheet_name=None)

In [12]:
def linear_stack(key_subwork):
    global key_subworks
    global dir_stack
    global info_basic
    global stainfo_all

    print("Collecting ",key_subwork,' ...')
    if os.path.exists(dir_stack+str(key_subwork)+'_gather_linear.h5'):
        print("File exists.")
        return

    time0 = time.time()

    nf = info_basic['nf']
    #f = info_basic['f']
    stainfo = stainfo_all[key_subwork]
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    names = []
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = np.min( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        idx2 = np.max( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        #idx1 = int(stalist_all.index(stalist[sta1]))
        #idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m +idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        count[i] = count_all[num]
        #r[i] = r0[num]
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])

    outname = str(key_subwork)+'_gather_linear.h5'
    if os.path.exists(dir_stack+outname):
        os.remove(dir_stack+outname)
    ncffile = h5py.File(dir_stack+outname,'w')
    
    ncffile.create_dataset('ncfs',data=ncfs_sum_linear)
    ncffile.create_dataset('r',data=r)
    ncffile.create_dataset('count',data=count)
    ncffile.create_dataset('f',data=f)
    ncffile.create_dataset('StationPairs',data=StationPairs)
    #print(ncffile.keys())
    ncffile.close()
    #np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.", str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))

In [13]:
nThreads = 60

In [14]:
for key_subwork in key_subworks:
    linear_stack(key_subwork)

Done in  0.11257123947143555  s. 1 / 682
Done in  0.36392903327941895  s. 2 / 682
Done in  0.39252686500549316  s. 3 / 682
Done in  0.44316768646240234  s. 4 / 682
Done in  0.4127514362335205  s. 5 / 682
Done in  0.5214388370513916  s. 6 / 682
Done in  0.943671464920044  s. 7 / 682
Done in  0.4808228015899658  s. 8 / 682
Done in  0.5715246200561523  s. 9 / 682
Done in  0.648076057434082  s. 10 / 682
Done in  0.7897264957427979  s. 11 / 682
Done in  0.949425220489502  s. 12 / 682
Done in  0.5621645450592041  s. 13 / 682
Done in  0.3843662738800049  s. 14 / 682
Done in  0.11981320381164551  s. 15 / 682
Done in  0.4332435131072998  s. 16 / 682
Done in  0.16272568702697754  s. 17 / 682
Done in  0.5806131362915039  s. 18 / 682
Done in  0.2407212257385254  s. 19 / 682
Done in  0.5449903011322021  s. 20 / 682
Done in  0.19485759735107422  s. 21 / 682
Done in  0.36933302879333496  s. 22 / 682
Done in  0.6954004764556885  s. 23 / 682
Done in  0.19604229927062988  s. 24 / 682
Done in  0.14292287

In [15]:
np.save(filename_bi,info_basic_bi)